In [2]:
# Instalação das bibliotecas
%pip install -U langgraph langsmith langchain_anthropic Flask twilio 

  Using cached flask-3.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached PyJWT-2.10.1-py3-none-any.whl.metadata (4.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 1.6 MB/s eta 0:00:0000:01
  Using cached MarkupSafe-3.0.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.0 kB)
Using cached flask-3.1.0-py3-none-any.whl (102 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.7 MB/s eta 0:00:00:00:0100:01
Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)
Using cached itsdangerous-2.2.

In [ ]:
# Disponibilizar dependências
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [9]:
from flask import Flask
app = Flask(__name__)

@app.route("/")
def hello():
  return "Hello World!"

if __name__ == "__main__":
  app.run()

 * Tip: There are .env files present. Install python-dotenv to use them.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Apr/2025 15:53:54] "GET / HTTP/1.1" 200 -


In [ ]:
# Inicialização do ambiente
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional
import random

# Definição do estado
class GuardiaoState(TypedDict):
    media = Optional[bytes]
    tipo = Optional[str]
    resultado_analise = Optional[dict]
    indice_confiabilidade = Optional[int]
    resposta = Optional[str]

# Função dos nós
def receber_input(state: GuardiaoState) -> GuardiaoState:
    """Recebe o input do usuário."""
    print("📥 Recebendo imagem/vídeo via WhatsApp...")  
    return{
        **state,
        "media": b"<simulated_binary_data>",
        "tipo": "video"
    }

def processar_midia(state: GuardiaoState) -> GuardiaoState:
    """Processa o input recebido."""
    print(f"🧪 Processando {state['tipo']}...")
    # Aqui você chamaria um modelo de deepfake detection (ex: HuggingFace, OpenCV, etc)
    resultado_falso = random.choice([True, False])
    explicacao = "Rosto inconsistente com movimento labial detectado." if resultado_falso else "Nenhuma manipulação aparente."
    return {
        **state,
        "resultado_analise": {
        "deepfake_detectado": resultado_falso,
        "explicacao": explicacao
        }
    }

def gerar_indice(state: GuardiaoState) -> GuardiaoState:
    """Gera um índice de confiabilidade."""
    print("📊 Calculando índice de confiabilidade...")
    confianca = random.randint(10,60) if state["resultado_analise"]["deepfake_detectado"] else random.randint(80, 100)
    indice = random.randint(0, 100)
    return {
        **state,
        "indice_confiabilidade": confianca
    }

def gerar_resposta(state: GuardiaoState) -> GuardiaoState:
    """Gera a resposta para o usuário."""
    print("📤 Gerando resposta para o usuário...")
    resposta = (
        f"⚠️ Possível manipulação detectada.\n"
        f"Índice de confiabilidade: {state['indice_confiabilidade']} / 100\n"
        f"Motivo: {state['resultado_analise']['explicacao']}"
        if state["indice_confiabilidade"] < 75
        else "✅ Mídia considerada confiável."
    )

    return {
        **state,
        "resposta": resposta
    }
    



In [ ]:
# Construção do grafo

workflow = StateGraph(GuardiaoState)

workflow.add_node("receber_input", receber_input)
workflow.add_node("processar_midia", processar_midia)
workflow.add_node("gerar_indice", gerar_indice)
workflow.add_node("gerar_resposta", gerar_resposta)

workflow.set_entry_point("receber_input")


workflow.add_edge("receber_input", "processar_media")
workflow.add_edge("processar_media", "gerar_indice")
workflow.add_edge("gerar_indice", "gerar_resposta")
workflow.add_edge("gerar_resposta", END)

graph = workflow.compile()

# Execução do grafo
estado_inicial = {}
final_state = graph.invoke(estado_inicial)


# Exibição do resultado
print("Resultado final:")
print(final_state["resposta"])

📥 Recebendo imagem/vídeo via WhatsApp...


KeyError: 'tipo'

## Próximos passos

In [ ]:
Card 1: Definição de Escopo
Monitorar exclusivamente mensagens de WhatsApp (via API: Twilio, Z‑API, Gupshup etc.).
Conteúdo alvo: imagens e vídeos.
Sinal de desinformação: deepfakes (rostos trocados, vozes sintetizadas, adulterações).
Output principal: índice de confiabilidade de 0 a 100.

Card 2: Coleta de Dados (Ingestão)
Configurar e autenticar bot/API do WhatsApp.
Receber mídia (imagens/vídeos) em um endpoint HTTP.
Armazenar temporariamente os arquivos num bucket ou diretório local.
Disparar evento para o pipeline de análise.

Card 3: Pré‑processamento de Mídia
Vídeo: extrair frames-chave (e.g. 1 frame a cada X segundos).
Imagem: redimensionar / normalizar resolução.
(Opcional) Extrair áudio se for detectar voz sintética.

Card 4: Detecção de Deepfake
Integrar modelo pré‑treinado (ex.: HuggingFace deepfake-detection, FaceForensics++).
Rodar inferência em cada frame/imagem: obter probabilidade de manipulação.
Agregar resultados por vídeo (e.g. média ou percentil de “suspeita”).

Card 5: Geração do Índice de Confiabilidade
Converter a probabilidade de deepfake num score de 0–100:
Exemplo: índice = 100 – int(probabilidade_deepfake * 100).
Definir thresholds (ex.: < 75 alerta, >= 75 confiável).
Registrar explicação resumida (e.g. “inconsistências nos lábios”).

Card 6: Resposta ao Usuário
Formatar mensagem de retorno via WhatsApp:
“⚠️ Vídeo suspeito. Índice: 22/100. Motivo: inconsistência nos lábios.”
“✅ Mídia confiável. Índice: 92/100.”
(Futuro) Extensão para X/Twitter com alertas semelhantes.

Card 7: Modo Educativo(opcional)
Criar fluxo opcional de quizzes rápidos:
Bot pergunta: “O que você observa neste frame adulterado?”
Feedback imediato com dicas de checagem de deepfake.
Agendar envios periódicos (catalogar por temas: rosto, áudio, metadata).

Card 8: Infraestrutura & Deploy(opcional)
Dockerizar cada componente (ingestão, pré-processamento, inferência, bot).
Orquestrar com Kubernetes / Docker Compose / LangGraph.
Monitorar logs e métricas (latência, taxa de falsos positivos).
Planejar credenciais e segurança (tokens das APIs, rate limits).

Card 9: Cronograma de 2 Semanas

Semana	Principais Entregas
1 (dias 1–4)	MVP local: upload manual + detecção + índice
1 (dias 5–7)	Orquestração com LangGraph + testes end‑to‑end
2 (dias 1–3)	Integração com WhatsApp bot (Twilio/Z‑API)
2 (dias 4–5)	Ajustes de UX do bot e implementação do modo educativo
2 (dias 6–7)	Polimento, documentação e apresentação final

In [ ]:
# Exemplo de enviar Imagem utilizando Z-Api

import requests

url = "https://api.z-api.io/instances/3E04FE2FFF3780EE584A02121E6AE94B/token/B4671208BBF614A2D90F157F/send-image"

payload = {
    "phone": "5516994420645",
    "image": "https://t.ctcdn.com.br/lvns56iaSMyHvyTur4JeYS_NYeY=/i606944.png"
}

headers = {'client-token': 'F51db6f26c0c94bbeb6131a2f18ca0638S', 
           'Content-Type': 'application/json'}

response = requests.post(url, headers=headers, json=payload)


print(response.text)

{"zaapId":"3E0503984A0810E6C3269ED4CE203EC0","messageId":"3EB062C78CC7A4584E6D41","id":"3EB062C78CC7A4584E6D41"}


In [41]:
# Exemplo de enviar Video utilizando Z-Api

import requests

url = "https://api.z-api.io/instances/3E04FE2FFF3780EE584A02121E6AE94B/token/B4671208BBF614A2D90F157F/send-video"

payload = {
    "phone": "5585982317976",
    "video": "https://files.catbox.moe/bbtrfk.mp4"
}

headers = {'client-token': 'F51db6f26c0c94bbeb6131a2f18ca0638S', 
           'Content-Type': 'application/json'}

response = requests.post(url, headers=headers, json=payload)


print(response.text)

{"zaapId":"3E05089EA56F60CC2C600AC1BE50C986","messageId":"3EB0BCB95C04C3F4BD5A20","id":"3EB0BCB95C04C3F4BD5A20"}
